## Code Setup

In [ ]:
# Path to main directory
DATA_PATH = "/content/gdrive/Shareddrives/Birds and CS/Data/CA-Final"
DATA_PATH = LOCAL_PATH if IS_LOCAL else REMOTE_PATH

In [ ]:
import os
import sys

In [ ]:
# Installs required packages
if not IS_LOCAL:
    !pip install ecoscape-connectivity
    !pip install ecoscape-utils

In [ ]:
# Connecting to Drive.
if not IS_LOCAL:
    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)

In [ ]:
import time
from ecoscape_utilities import BirdRun
from ecoscape_connectivity import compute_connectivity
from ecoscape_connectivity.util import read_transmission_csv

## Bird Run Definition

In [ ]:
bird_run = BirdRun(DATA_PATH)

def create_bird_runs(target):
    """Creates bird runs for the specified output target."""
    birds = []

    birds.append(bird_run.get_bird_run(
        "acowoo", "Acorn Woodpecker", run_name=target)

    birds.append(bird_run.get_bird_run(
        "stejay", "Steller's Jay", run_name=target)

    for bird in birds:

        # Creates output folder, if missing.
        bird_run.createdir_for_file(bird.repopulation_fn)
        bird_run.createdir_for_file(bird.gradient_fn)

    return birds


## Alternate connectivity algorithms

This is the standard EcoScape algorithm.

In [ ]:
class StochasticRepopulateFast(nn.Module):
    """
    Important: THIS is the function to use in the repopulation experiments.
    This module models the repopulation of the habitat from a chosen percentage
    of the seed places.  The terrain and habitat are parameters, and the input is a
    similarly sized 0-1 (float) tensor of seed points."""

    def __init__(self, habitat, terrain, num_spreads=100, spread_size=1, min_transmission=0.9,
                 randomize_source=True, randomize_dest=False):
        """
        :param habitat: torch tensor (2-dim) representing the habitat.
        :param terrain: torch tensor (2-dim) representing the terrain.
        :param num_spreads: number of bird spreads to use
        :param spread_size: by how much (in pixels) birds spread
        :param min_transmission: min value used in randomizations.
        :param randomize_source: whether to randomize the source of the spread.
        :param randomize_dest: whether to randomize the destination of the spread.
        """
        super().__init__()
        self.habitat = habitat
        self.goodness = torch.nn.Parameter(torch.max(habitat, terrain), requires_grad=True)
        self.h, self.w = habitat.shape
        self.num_spreads = num_spreads
        self.spread_size = spread_size
        # Defines spread operator.
        self.min_transmission = min_transmission
        self.randomize_source = randomize_source
        self.randomize_dest = randomize_dest
        self.kernel_size = 1 + 2 * spread_size
        self.spreader = torch.nn.MaxPool2d(self.kernel_size, stride=1, padding=spread_size)


    def forward(self, seed):
        """
        seed: a 0-1 (float) tensor of seed points.
        """
        # First, we multiply the seed by the habitat, to confine the seeds to
        # where birds can live.
        x = seed * self.habitat
        if x.ndim < 3:
            # We put it into shape (1, w, h) because the pooling operator expects this.
            x = torch.unsqueeze(x, dim=0)
        # Now we must propagate n times.
        for _ in range(self.num_spreads):
            # First, we randomly suppress some bird origin locations.
            xx = x
            if self.randomize_source:
                x = x * (self.min_transmission + (1. - self.min_transmission) * torch.rand_like(x))
            # Then, we propagate.
            x = self.spreader(x) * self.goodness
            # We randomize the destinations too.
            if self.randomize_dest:
                x *= (self.min_transmission + (1. - self.min_transmission) * torch.rand_like(x))
            # And finally we combine the results.
            x = torch.max(x, xx)
        x *= self.habitat
        if seed.ndim < 3:
            x = torch.squeeze(x, dim=0)
        return x

    def get_grad(self):
        return self.goodness.grad * self.goodness

This is the algorithm that uses patch size as a measure of connectivity.

In [ ]:
def analyze_tile_connected_patches(device=None):
    """This is the function that performs the analysis on a single tile.
    The input and output to this function are in the cpu, but the computation occurs in
    the specified device.
    str device: the device to be used, either cpu or cuda.
    """

    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')


    def f(habitat, terrain):
        _, w, h = habitat.shape
        # This is the function, which returns the connectivity as the size of a habitat patch.

        hab = torch.tensor(habitat.astype(np.float), requires_grad=False, dtype=torch.float, device = device).view(w, h)
        ter = torch.tensor(terrain.astype(np.float), requires_grad=False, dtype=torch.float, device = device).view(w, h)
        repopulator = analysis_class(hab, ter, num_spreads=total_spreads, spread_size=hop_length).to(device)
        for i in range(num_batches):
            # Creates the seeds.
            seeds = torch.rand((batch_size, w, h), device=device) < seed_probability
            # And passes them through the repopulation.
            pop = repopulator(seeds)
            # We need to take the mean over each batch.  This will tell us what is the
            # average repopulation.
            tot_pop += torch.mean(pop, 0)
            # This is the sum across all batches.  So, the gradient will be for the total
            # of the batch. This is why the gradient will need to be divided by the number
            # of simulations.
            if produce_gradient:
                q = torch.sum(pop)
                q.backward()
                tot_grad += repopulator.get_grad()
        # Normalizes by number of batches.
        avg_pop, avg_grad = tot_pop / num_batches, tot_grad / num_simulations
        return avg_pop.to("cpu"), avg_grad.to("cpu")
    if not produce_gradient:
        # We remove all memory/time requirements due to gradient computation.
        f = torch.no_grad()(f)
    return f


In [ ]:
class PatchArea(nn.Module):
    """
    This computes connectivity as the area of habitat patches.
    """

    def __init__(self, habitat, terrain, num_spreads=100, spread_size=1, min_transmission=0.9,
                 randomize_source=True, randomize_dest=False):
        """
        :param habitat: torch tensor (2-dim) representing the habitat.
        :param terrain: torch tensor (2-dim) representing the terrain. Not used.
        :param num_spreads: Not used.
        :param spread_size: Not used.
        :param min_transmission: Not used.
        :param randomize_source: Not used.
        :param randomize_dest: Not used.
        """
        super().__init__()
        self.habitat = habitat


    def forward(self, seed):
        """
        seed: a 0-1 (float) tensor of seed points.
        """
        # First, we multiply the seed by the habitat, to confine the seeds to
        # where birds can live.
        x = seed * self.habitat
        if x.ndim < 3:
            # We put it into shape (1, w, h) because the pooling operator expects this.
            x = torch.unsqueeze(x, dim=0)
        # Now we must propagate n times.
        for _ in range(self.num_spreads):
            # First, we randomly suppress some bird origin locations.
            xx = x
            if self.randomize_source:
                x = x * (self.min_transmission + (1. - self.min_transmission) * torch.rand_like(x))
            # Then, we propagate.
            x = self.spreader(x) * self.goodness
            # We randomize the destinations too.
            if self.randomize_dest:
                x *= (self.min_transmission + (1. - self.min_transmission) * torch.rand_like(x))
            # And finally we combine the results.
            x = torch.max(x, xx)
        x *= self.habitat
        if seed.ndim < 3:
            x = torch.squeeze(x, dim=0)
        return x

    def get_grad(self):
        return self.goodness.grad * self.goodness

In [ ]:
import torch
import numpy as np

In [ ]:
# prompt: get the shape of a tensor in pytorch

a = torch.zeros((4, 6))


In [ ]:
list(a.shape)

[4, 6]

In [ ]:
ix, iy = np.mgrid[:4, :6]

In [ ]:
ix

array([[0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3]])

In [ ]:
iy

array([[6]])